In [8]:
import os
os.chdir("/home/kaborg15/MapAI-Competition/team_morty/src")
os.listdir("/home/kaborg15/Stable_Diffusion/MapAI_Generated_images/1-5_inpainting_rooftops/images")

['6123_532_75_4.jpg',
 '6078_637_36_1.jpg',
 '6107_577_68_3.jpg',
 '6188_599_21_0.jpg',
 '6123_531_46_1.jpg',
 '6078_638_65_4.jpg',
 '6088_491_13_2.jpg',
 '6172_727_40_2.jpg',
 '6248_475_1_3.jpg',
 '6259_563_44_0.jpg',
 '6234_553_19_1.jpg',
 '6147_482_27_2.jpg',
 '6170_651_86_3.jpg',
 '6147_482_1_1.jpg',
 '6165_679_64_0.jpg',
 '6248_476_73_4.jpg',
 '6227_534_30_2.jpg',
 '6173_631_66_3.jpg',
 '6220_509_45_2.jpg',
 '6141_649_33_1.jpg',
 '6165_679_71_3.jpg',
 '6170_652_33_1.jpg',
 '6084_553_61_2.jpg',
 '6112_695_60_0.jpg',
 '6215_476_52_0.jpg',
 '6104_604_30_1.jpg',
 '6171_727_76_0.jpg',
 '6220_509_13_4.jpg',
 '6190_693_52_4.jpg',
 '6188_550_99_3.jpg',
 '6230_605_52_1.jpg',
 '6259_563_8_1.jpg',
 '6107_577_74_1.jpg',
 '6104_603_22_2.jpg',
 '6088_492_45_3.jpg',
 '6248_476_18_4.jpg',
 '6122_520_66_0.jpg',
 '6230_604_63_0.jpg',
 '6179_495_22_0.jpg',
 '6088_491_80_1.jpg',
 '6104_604_30_4.jpg',
 '6122_519_39_3.jpg',
 '6173_630_90_2.jpg',
 '6122_520_46_0.jpg',
 '6172_727_41_2.jpg',
 '6078_638_20

In [1]:
!nvidia-smi

Tue Nov  8 07:57:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:E0:00.0 Off |                    0 |
| N/A   32C    P0    67W / 350W |  12191MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM3...  On   | 00000000:E2:00.0 Off |                    0 |
| N/A   

In [3]:
! pip install opencv-python==4.5.5.64
#! pip install opencv-python --upgrade

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 60.5 MB 4.0 MB/s eta 0:00:01
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.6.0.66
    Uninstalling opencv-python-4.6.0.66:
      Successfully uninstalled opencv-python-4.6.0.66


In [26]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 441 kB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 52.7 MB/s eta 0:00:01
     |████████████████████████████████| 212 kB 42.9 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 81.9 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 53.0 MB/s eta 0:00:01
     |████████████████████████████████| 35.3 MB 44.4 MB/s eta 0:00:01
     |████████████████████████████████| 262 kB 41.0 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 52.8 MB/s eta 0:00:01
     |████████████████████████████████| 161 kB 85.3 MB/s eta 0:00:01
     |████████████████████████████████| 131 kB 80.0 MB/s eta 0:00:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 5.0.0
    Uninstalling pyarrow-5.0.0:
      Successfully uninstalled pyarrow-5.0.0


In [4]:

from yaml import load, dump, Loader, Dumper
import argparse
from augmentation.dataloader import create_dataloader

DATA_FOLDER_DICT = {
    "original_images": "../../data/train/images/",
    "original_masks": "../../data/train/masks/",
    "generated_data_folders": ["1-5_inpainting_rooftops/images"],
    "generated_data_root": "/home/kaborg15/Stable_Diffusion/MapAI_Generated_images/",
    "augment_data_mode": "append" # Original, append, replace
}

import os
import wandb
LOG_WANDB = False
if LOG_WANDB:
    wandb.init(project="MapAi-train")
wandb.config = {
    "epochs": 40,
    "learning_rate": 5e-5,
    "batch_size": 8,
    "task": 1
}


parser = argparse.ArgumentParser("Training a segmentation model")

parser.add_argument("--epochs", type=int, default=wandb.config["epochs"], help="Number of epochs for training")
parser.add_argument("--lr", type=float, default=wandb.config["learning_rate"], help="Learning rate used during training")
parser.add_argument("--config", type=str, default="config/data.yaml", help="Configuration file to be used")
parser.add_argument("--device", type=str, default="cuda:2")
parser.add_argument("--task", type=int, default=wandb.config["task"])
parser.add_argument("--data_ratio", type=float, default=1.0,
                    help="Percentage of the whole dataset that is used")
#parser.add_argument("--resume", type=str, default="runs/task_1/run_116/last_task1_199.pt", help="Path to state dict to resume training from, default is None")
parser.add_argument("--resume", type=str, default=None, help="Path to state dict to resume training from, default is None")
parser.add_argument("--datasets", type=dict, default=DATA_FOLDER_DICT, help="Path to original dataset")
parser.add_argument("--batch_size", type=int, default=wandb.config["batch_size"], help="Batch size used during training")
parser.add_argument("--prefix", type=str, default="", help="Prefix for run name")

class Dummy_Args:
    prefix = "test_args"
    config = "config/data.yaml"
    task = 1

args = Dummy_Args()
# Import config
opts = load(open(args.config, "r"), Loader)

# Combine args and opts in single dict
try:
    opts = opts | vars(args)
except Exception as e:
    opts = {**opts, **vars(args)}

print("Overriding config batch size with wandb config batch size")
#opts[f"task{opts['task']}"]["batchsize"] = wandb.config["batch_size"]
print("Opts:", opts)

ImportError: libGL.so.1: cannot open shared object file: No such file or directory

In [24]:
import os
import re
ORIGINAL_DATASET_FOLDER = "D:/data/train/images"
DATASET_ROOT_FOLDER = "team_morty/Stable Diffusion Augmentation/"
DATASET_FOLDERS = ["1-5_inpainting_rooftops/images", "generated_dataset_1-5/images"]
MASKS_FOLDER_PATH = "D:/data/train/masks"

#DATASET_FOLDER = "team_morty/Stable Diffusion Augmentation/generated_dataset_1-5/images"
os.chdir("D:/PycharmProjects/MapAI-Competition")

In [17]:
example_file_name = os.listdir(ORIGINAL_DATASET_FOLDER)[42]
#original_file_name = "_".join(example_file_name.split("_")[:-1])
original_file_name = example_file_name.split(".")[0]
original_file_name

'6051_689_47'

In [39]:
def preprocess_dataset_folders(dataset_folders, dataset_root):
    file_iterations_dict = {}
    for dataset_folder in dataset_folders:
        for file_name in os.listdir(os.path.join(dataset_root, dataset_folder)):
            file_name_base = "_".join(file_name.split("_")[:-1])
            if file_name_base not in file_iterations_dict:
                file_iterations_dict[file_name_base] = []
            file_iterations_dict[file_name_base].append(os.path.join(dataset_folder, file_name))
    return file_iterations_dict

def draw_dataset_files(dataset_folders, dataset_root, original_dataset_folder, original_mask_folder):
    preprocessed_files = preprocess_dataset_folders(dataset_folders, dataset_root)
    selected_paths = []
    for original_file in os.listdir(original_dataset_folder):
        original_file_base = original_file.split(".")[0]
        file_iterations = preprocessed_files[original_file_base]
        drawn_file = choice(file_iterations)
        selected_paths.append({"image": os.path.join(dataset_root,drawn_file), "mask": os.path.join(original_mask_folder,drawn_file)})
    return selected_paths

len(draw_dataset_files(DATASET_FOLDERS, DATASET_ROOT_FOLDER, ORIGINAL_DATASET_FOLDER, MASKS_FOLDER_PATH))

7000

In [ ]:
for original_file in os.listdir(ORIGINAL_DATASET_FOLDER):
    original_file_base = original_file.split(".")[0]
    file_iterations = preprocessed_files[original_file_base]
    drawn_file = choice(file_iterations)
    selected_paths.append(os.path.join(DATASET_ROOT_FOLDER, drawn_file))

In [ ]:
selected_paths = []
for original_file in os.listdir(ORIGINAL_DATASET_FOLDER):
    original_file_base = original_file.split(".")[0]
    file_iterations = preprocessed_files[original_file_base]
    drawn_file = choice(file_iterations)
    selected_paths.append(os.path.join(DATASET_ROOT_FOLDER, drawn_file))

In [ ]:
elected_paths = []
for original_file in os.listdir(ORIGINAL_DATASET_FOLDER):
    original_file_base = original_file.split(".")[0]
    file_iterations = preprocessed_files[original_file_base]
    drawn_file = choice(file_iterations)
    selected_paths.append(os.path.join(DATASET_ROOT_FOLDER, drawn_file))

In [29]:
from random import choice

def draw_random_iteration(original_file_name, dataset_folders, root_folder):
    file_search_pattern = re.compile(f"{original_file_name}_\d+.jpg")
    matching_files = []
    for folder in dataset_folders:
        for file in os.listdir(os.path.join(root_folder,folder)):
            if not file_search_pattern.match(file):
                continue
            file_path = os.path.join(folder, file)
            matching_files.append(file_path)

    return choice(matching_files)

def construct_psuedo_paths_draw(original_dataset_folder, dataset_folders, dataset_root_folder):
        file_names = os.listdir(original_dataset_folder)
        paths = []
        for file_name in file_names:
            if file_name == ".ipynb_checkpoints":
                continue
            original_file_name = file_name.split(".")[0]
            drawn_file = draw_random_iteration(original_file_name, dataset_folders, dataset_root_folder)
            paths.append({"image": os.path.join(dataset_root_folder,file_name), "mask": os.path.join(MASKS_FOLDER_PATH,file_name)})
        return paths

construct_psuedo_paths_draw(ORIGINAL_DATASET_FOLDER, DATASET_FOLDERS, DATASET_ROOT_FOLDER)

[{'image': 'team_morty/Stable Diffusion Augmentation/6051_689_0.tif',
  'mask': 'D:/data/train/masks\\6051_689_0.tif'},
 {'image': 'team_morty/Stable Diffusion Augmentation/6051_689_1.tif',
  'mask': 'D:/data/train/masks\\6051_689_1.tif'},
 {'image': 'team_morty/Stable Diffusion Augmentation/6051_689_10.tif',
  'mask': 'D:/data/train/masks\\6051_689_10.tif'},
 {'image': 'team_morty/Stable Diffusion Augmentation/6051_689_11.tif',
  'mask': 'D:/data/train/masks\\6051_689_11.tif'},
 {'image': 'team_morty/Stable Diffusion Augmentation/6051_689_12.tif',
  'mask': 'D:/data/train/masks\\6051_689_12.tif'},
 {'image': 'team_morty/Stable Diffusion Augmentation/6051_689_13.tif',
  'mask': 'D:/data/train/masks\\6051_689_13.tif'},
 {'image': 'team_morty/Stable Diffusion Augmentation/6051_689_14.tif',
  'mask': 'D:/data/train/masks\\6051_689_14.tif'},
 {'image': 'team_morty/Stable Diffusion Augmentation/6051_689_15.tif',
  'mask': 'D:/data/train/masks\\6051_689_15.tif'},
 {'image': 'team_morty/Stabl

In [15]:
def get_image_iteration_numbers(folder, file_name):
    pattern = re.compile(f"{file_name}_([0-9]+).jpg")
    # Find all files that match the pattern in the folder
    files = os.listdir(folder)
    files = [file for file in files if pattern.match(file)]
    return len(files)

import random

def draw_augmented_image(folder, file_name, n_images):
    iteration_number = random.randint(0, n_images)
    image_path = os.path.join(folder, f"{file_name}_{iteration_number}.jpg")
    return image_path

def construct_psuedo_paths_draw(original_dataset_folder, dataset_folder):
        file_names = os.listdir(f"{data_path}")
        image_iterations = get_image_iteration_numbers()
        paths = []
        for file_name in file_names:
            if file_name == ".ipynb_checkpoints":
                continue
            iteration_number = self.get_file_iteration_number(file_name)
            if size_reducer != -1 and iteration_number > size_reducer:
                continue
            if iteration_number == 0 and not keep_original:
                continue
            original_file_name = self.get_original_file_name(file_name)
            paths.append({"image": f"{data_path}/{file_name}", "mask": f"{MASKS_FOLDER_PATH}/{original_file_name}"})
        
        return paths

get_image_iteration_numbers(DATASET_FOLDER, original_file_name)

10